
# XML für die Korpuslinguistik — Workshop (90 Minuten)

**Dozent**: Phillip B. Ströbel (UZH)  
**Ziele**: In 90 Minuten weisst du, …  
- was XML ist und warum es sich in der Korpuslinguistik bewährt,  
- was Header/Body bedeutet (z. B. TEI),  
- wie Tags/Attribute/Hierarchien funktionieren,  
- was der Unterschied zwischen flachem und tiefem XML ist,  
- welche Python-Bibliotheken sich für XML eignen und wie man einfache Analysen macht.

**Material**: Dieses Notebook findest du unter https://pstroe.github.io/xml-workshop-zukoko-2025/lab/index.html. Die Daten im Ordner `data/`.



## 0) Setup
Wir verwenden primär die Standardbibliothek `xml.etree.ElementTree` (kurz: `ET`). Falls verfügbar, nutzen wir optional `lxml` für XPath.


In [ ]:

# Imports & Feature-Flags
import os
from pathlib import Path
import xml.etree.ElementTree as ET

# Optional: lxml (für volle XPath-Unterstützung)
try:
    import lxml.etree as LET
    USING_LXML = True
except Exception as e:
    USING_LXML = False

DATA_DIR = Path("data")
print("USING_LXML =", USING_LXML)
print("DATA_DIR =", DATA_DIR.resolve())



## 1) Erste Beispiele: TEI und Gesprächs-Korpus
Wir schauen uns zwei Dateien an:
- `tei_sample.xml` (Header/Body-Struktur)  
- `corpus_small.xml` (Dialog mit `<u>`, `<s>`, `<w>` und `<pc>`)


In [ ]:

# Datei laden und Root inspizieren (ElementTree)
tei_path = DATA_DIR / "tei_sample.xml"
tree = ET.parse(tei_path)
root = tree.getroot()
print("Root-Tag (mit Namespace):", root.tag)
print("Kinder:", [child.tag for child in list(root)[:3]])


In [ ]:

# Dialog-Korpus laden und erste Tokens ausgeben
corpus_path = DATA_DIR / "corpus_small.xml"
ctree = ET.parse(corpus_path)
croot = ctree.getroot()

for u in croot.findall(".//u"):
    speaker = u.attrib.get("who")
    words = [ (w.text or "") for w in u.findall(".//w") ]
    print(speaker, ":", " ".join(words))



## 2) Tags, Attribute, Hierarchie kurz & bündig
- **Elemente** (Tags): `<w>Gehe</w>`  
- **Attribute**: `<w lemma="gehen" pos="VVFIN">Gehe</w>`  
- **Hierarchie**: `<u>` enthält `<s>`, `<s>` enthält `<w>` und `<pc>`.

👉 **Mini-Aufgabe:** Hole alle Wortformen (`<w>`) von Sprecher `#A` und gib sie als Liste aus.


In [ ]:

# Lösungsvorschlag
words_a = []
for u in croot.findall(".//u"):
    if u.attrib.get("who") == "#A":
        words_a.extend([ (w.text or "") for w in u.findall(".//w") ])

words_a



## 3) Flaches vs. tiefes XML
- **Flach**: Informationen als Attribute in höherer Ebene (z. B. alle Tokens als String-Attribut)
- **Tief**: Explizite Verschachtelung mit Unterelementen

Beispieldateien:
- `corpus_flat.xml` (flach)
- `corpus_small.xml` (tief)

👉 **Aufgabe:** Parse `corpus_flat.xml` und splitte die Tokens pro Sprecher.


In [ ]:

# Lösungsvorschlag
flat_path = DATA_DIR / "corpus_flat.xml"
ftree = ET.parse(flat_path)
froot = ftree.getroot()

for u in froot.findall(".//u"):
    speaker = u.attrib.get("who")
    toks = (u.attrib.get("tokens") or "").split("|")
    toks = [t.strip() for t in toks if t.strip()]
    print(speaker, ":", toks)



## 4) Token-Frequenzen, Lemmata und POS
Wir extrahieren Tokens (`<w>`) und zählen Frequenzen. Dann fassen wir nach Lemma und POS zusammen.


In [ ]:

from collections import Counter
import pandas as pd

def iter_tokens(root):
    for w in root.findall(".//w"):
        yield (w.text or ""), w.attrib.get("lemma"), w.attrib.get("pos")

rows = [(form, lemma, pos) for form, lemma, pos in iter_tokens(croot)]
df = pd.DataFrame(rows, columns=["form", "lemma", "pos"])
df


In [ ]:

# Häufigkeiten der Wortformen
form_counts = df["form"].value_counts().reset_index()
form_counts.columns = ["form", "freq"]
form_counts


In [ ]:

# Gruppierung nach Lemma und POS
lemma_pos_counts = df.groupby(["lemma", "pos"]).size().reset_index(name="freq")
lemma_pos_counts.sort_values("freq", ascending=False)



## 5) (Optional) Visualisierung der Top-Token
Ein einfacher Balkenplot der häufigsten Formen.


In [ ]:

import matplotlib.pyplot as plt

topn = 10
top_forms = form_counts.head(topn)

plt.figure()
plt.bar(top_forms["form"], top_forms["freq"])  # keine Farben explizit setzen
plt.title(f"Top {topn} Wortformen")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()



## 6) XPath (mit lxml, falls installiert)
Mit `lxml` können wir volle XPath-Queries ausführen. Beispiel: alle finiten Verben (`pos="VVFIN"`) von Sprecher `#A`.


In [ ]:

if USING_LXML:
    # lxml-Parsing
    ltree = LET.parse(str(corpus_path))
    lroot = ltree.getroot()
    # XPath-Query
    nodes = lroot.xpath(".//u[@who='#A']//w[@pos='VVFIN']")
    [n.text for n in nodes]
else:
    print("lxml ist nicht verfügbar. Überspringe XPath-Beispiel.")



## 7) Schreiben und Modifizieren von XML
Wir fügen eine einfache Normalisierung hinzu: Attribut `norm` für Tokens in Grossbuchstaben (`text.upper()`).


In [ ]:

# Kopie laden, norm hinzufügen und speichern
tree2 = ET.parse(corpus_path)
root2 = tree2.getroot()

for w in root2.findall(".//w"):
    if w.text:
        w.set("norm", w.text.upper())

out_path = DATA_DIR / "corpus_annotated.xml"
tree2.write(out_path, encoding="utf-8", xml_declaration=True)
print("Gespeichert:", out_path.resolve())



## 8) Well-formedness & Fehlersuche
Wir versuchen, `broken.xml` zu parsen und fangen den Fehler ab.


In [ ]:

broken_path = DATA_DIR / "broken.xml"
try:
    ET.parse(broken_path)
    print("Kein Fehler erkannt (unerwartet).")
except ET.ParseError as e:
    print("ParseError:", e)



## 9) Übungen
**Übung 1 (5–10 Min):**  
Konvertiere `dialogue.txt` zu einfachem XML mit `<u who>`, `<s>`, `<w>`.  
Tipp: Splitte Zeilen nach `":"` (Sprecher) und whitespace (Tokens).

**Übung 2 (10 Min):**  
Berechne für `corpus_small.xml` die **Type-Token-Ratio (TTR)** pro Sprecher.

**Übung 3 (10–15 Min):**  
Erzeuge **Bigrams** (Folgepaare) der Wortformen pro Sprecher und zähle die häufigsten.

**Bonus (falls Zeit):**  
- Füge eine neue Annotation `sentiment="pos/neg/neu"` zu allen Tokens hinzu (dummy: pos für A, neu für B).  
- Exportiere eine Token-Tabelle als CSV (Form, Lemma, POS, Sprecher).


In [ ]:

# Starte hier mit Übung 1:
# 1) Lade dialogue.txt, 2) parse die Zeilen, 3) baue ein XML-Baumobjekt, 4) speichere als dialogue.xml

from xml.etree.ElementTree import Element, SubElement, ElementTree

dlg_in = (DATA_DIR / "dialogue.txt").read_text(encoding="utf-8").splitlines()

root = Element("corpus", {"xml:lang":"de"})
sid = 1
for line in dlg_in:
    if ":" not in line:
        continue
    speaker, text = line.split(":", 1)
    speaker = speaker.strip().replace("Speaker ", "#")
    u = SubElement(root, "u", {"who": speaker})
    s = SubElement(u, "s", {"n": str(sid)})
    sid += 1
    # sehr simple Tokenisierung
    for tok in text.strip().split():
        if tok in [".", ",", "!", "?", ";", ":"]:
            pc = SubElement(s, "pc")
            pc.text = tok
        else:
            w = SubElement(s, "w")
            w.text = tok

out_xml = DATA_DIR / "dialogue.xml"
ElementTree(root).write(out_xml, encoding="utf-8", xml_declaration=True)
print("dialogue.xml gespeichert nach:", out_xml.resolve())


In [ ]:

# Übung 2: TTR pro Sprecher (Lösungsvorschlag)

def ttr_for_speaker(root, who):
    tokens = [ (w.text or "") for w in root.findall(f".//u[@who='{who}']//w") ]
    types = set(tokens)
    return len(types) / max(1, len(tokens))

root_dialogue = ET.parse(DATA_DIR / "dialogue.xml").getroot()
speakers = sorted({u.attrib.get("who") for u in root_dialogue.findall(".//u")})

for spk in speakers:
    print(spk, "TTR =", round(ttr_for_speaker(root_dialogue, spk), 3))


In [ ]:

# Übung 3: Bigrams zählen (Lösungsvorschlag)

from collections import Counter

def bigrams(seq):
    for i in range(len(seq)-1):
        yield (seq[i], seq[i+1])

def speaker_bigrams(root, who):
    toks = [ (w.text or "") for w in root.findall(f".//u[@who='{who}']//w") ]
    return Counter(bigrams(toks))

for spk in speakers:
    bg = speaker_bigrams(root_dialogue, spk).most_common(10)
    print("\n", spk, "Top-10 Bigrams:")
    for (a, b), f in bg:
        print(f"{a} {b} -> {f}")



## 10) Export: Token-Tabelle als CSV
Wir schreiben eine Tokenliste (Form, Lemma, POS, Sprecher) aus `corpus_small.xml` als CSV.


In [ ]:

rows = []
for u in croot.findall(".//u"):
    spk = u.attrib.get("who")
    for w in u.findall(".//w"):
        rows.append({
            "speaker": spk,
            "form": w.text or "",
            "lemma": w.attrib.get("lemma"),
            "pos": w.attrib.get("pos")
        })

df_tokens = pd.DataFrame(rows)
csv_path = DATA_DIR / "corpus_tokens.csv"
df_tokens.to_csv(csv_path, index=False, encoding="utf-8")
csv_path



## 11) Ausblick & Ressourcen
- **Standards**: TEI, EpiDoc, ParlaMint  
- **Validierung**: RELAX NG / Schematron (meist mit oXygen oder CI-Workflows)  
- **Python**: `xml.etree.ElementTree` für Basics, `lxml` für XPath/XSLT
- **Weiteres**: Export zu CoNLL/CSV, Alignments, UD, etc.

Viel Erfolg beim Ausprobieren!
